In [1]:
import sys
sys.path.append('../../NLP_analysis/')

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import datasets, layers, models

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

In [3]:
dataset = pd.read_csv('../Data/clean_data.csv')
dataset

,Title,Category
0,chainlink link falter hedera hbar wobbl yet vc...,Business
1,fund punish own nvidia share stun 230 ralli,Business
2,crude oil price stall hedg fund sell kemp,Business
3,grayscal bitcoin win still half battl,Business
4,im home shop editor cantmiss labor day deal im...,Business
...,...,...
51297,slovakia elect could echo ukrain here expect,Worldwide
51298,thing know nobel prize washington post,Worldwide
51299,brief calm protest kill 2 student rock imphal,Worldwide
51300,‘ no one safe ’ franc vow action bedbug sweep ...,Worldwide


In [15]:
label_encoder = LabelEncoder()
dataset['Category_enc'] = label_encoder.fit_transform(dataset['Category'])
labels = dataset['Category_enc']

In [22]:
labels.unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [37]:
dataset['Title'].head()

0    chainlink link falter hedera hbar wobbl yet vc...
1          fund punish own nvidia share stun 230 ralli
2            crude oil price stall hedg fund sell kemp
3                grayscal bitcoin win still half battl
4    im home shop editor cantmiss labor day deal im...
Name: Title, dtype: object

In [41]:
# Define hyperparameters
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 64

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Title'])
sequences = tokenizer.texts_to_sequences(dataset['Title'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating='post', padding='post')

In [42]:
model = Sequential([
    Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 100, 64)           640000    
                                                                 
 lstm_16 (LSTM)              (None, 100, 64)           33024     
                                                                 
 dropout_12 (Dropout)        (None, 100, 64)           0         
                                                                 
 lstm_17 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_13 (Dropout)        (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 64)                2112      
                                                                 
 dense_17 (Dense)            (None, 8)               

In [40]:
train_sequences, val_sequences, train_labels, val_labels = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

history = model.fit(train_sequences, train_labels, validation_data=(val_sequences, val_labels), epochs=10, batch_size=32)

Epoch 1/10
 275/1283 [=====>........................] - ETA: 45s - loss: 2.0709 - accuracy: 0.1435

KeyboardInterrupt: 